13.付録A: 日経225miniのデータ処理(CQG)

　CQGの日経225mini先物の標品コードはF.US.MJNKで、データは一か月分をフォルダーに入れて配布される。2018年12月であれば、フォルダー名はF.US.MJNK201812となる。その中には限月ごとのデータが入っている。その限月はF.US.MJNKのつぎに記載され、2019年３月限りではH19である。Hは取引所の限月コードである。したがって、ファイル名は
 
 F.US.MJNKH19_201812.ts
 
となる。tsはファイルを識別する拡張子である。弦月コードを表Aにまとめた。

|表A:|弦月コード|||||
|:-:|:-:|:-:|:-:|:-:|:-:|
|月|コード|月|コード|月|コード|
|1月|F|2月|G|3月|H|
|4月|J|5月|K|6月|M|
|7月|N|8月|Q|9月|U|
|10月|V|11月|X|12月|Z|
 
 オリジナルデータのフォーマットは
 
 1. 商品コード
 2. 取引日:YYYYMMDD
 3. 立会コード:0=第一、1=第二,2=第三
 4. 取引時間
 5. 取引価格
 6. 取引タイプ：B=買い,A=売り,T=取引,S=決済
 7. 市場の指標：F=Fast market, N=Normal market
 8. 取引訂正コード
 9. 取引高

であるので、これを

Date Price BestBid BestAsk Volume BBV BAV

に変更する。Priceは約定価格であり、BestBid、BestAskはそのときの最良買指値と売り指値である。Volumeは取引高であり、BBV,BAVは指値の提示数量である。

入力[1]は必要なライブライ－のインポート

In [1]:
import csv
from dateutil import parser as dp
from datetime import datetime, date
import pandas as pd

In [2]:
def CQGfile_convert_csv(file_name_read,file_name_save):
    n=0
    t=0
    trades=[]
    da=[]
    bestbid=0
    bestbidv=0
    bestask=0
    bestaskv=0
    with open(file_name_read,'r') as f:
        series=csv.reader(f)
        for line in series:
            i=0
            tmp=[0]*9
            for elements in line:
                tmp[i]=elements
                i+=1
            if tmp[5]=="B":
                bestbid=tmp[4]
                bestbidv=tmp[8]
            if tmp[5]=="A":
                bestask=tmp[4]
                bestaskv=tmp[8]
            if tmp[5]=="T":
                yymmdd=tmp[1]
                hm=tmp[3]
                da.append(datetime(int(yymmdd[0:4]),int(yymmdd[4:6]),
                                   int(yymmdd[6:8]),int(hm[0:2]),int(hm[2:4])))
                trades.append([])
                trades[t].append(tmp[4])
                trades[t].append(bestbid)
                trades[t].append(bestask)
                trades[t].append(tmp[8])
                trades[t].append(bestbidv)
                trades[t].append(bestaskv)
                t=t+1
            if tmp[4]==0:
                print(tmp[1],tmp[3])
            n=n+1
        print(file_name_read)
        print("total data points:",n,"no. of trades: ",t,\
              "% 2.4f"%(float(t)/float(n)))
        f.close()
    tsd=pd.DataFrame(trades,index=da,\
                columns=['Price','BestBid','BestAsk','Volume','BBV','BAV'])
    tsd.index.name='Date'
    tsd=tsd.tz_localize('US/Central')    
    tsd=tsd.tz_convert('Asia/Tokyo')    
    tsd.to_csv(file_name_save)

In [3]:
if __name__ == "__main__":
    t1=datetime.now()
    folders=["F.US.MJNK_201812"
    ]
    filenames0=["F.US.MJNKH19_201812.ts"
    ]
    filenames=["n225m201812Mar.csv"
    ]
    buf_path0="C:/users/xxx/documents/Database/n225/tick/"
    for folder,filename0,filename in zip(folders,filenames0,filenames):
        file_name_read=buf_path0+folder+"/"+filename0
        file_name_save=buf_path0+filename
        CQGfile_convert_csv(file_name_read,file_name_save)
print("---------------------------------------------------------")
t2=datetime.now()
print(t2-t1)

C:/users/hm201/documents/Database/n225/tick/F.US.MJNK_201812/F.US.MJNKH19_201812.ts
total data points: 28687059 no. of trades:  3801640  0.1325
---------------------------------------------------------
0:01:25.424841


13.付録B: 日経225miniの立会４本値の作成

　入力[3]で出力されたファイルを読み込み、立会４本値を作成する。フォーマットは

Date,Open,High,Low,Close,Volume,TradingValue

である。時間の表示は日中立会を9:00、夜間立会を16:30に統一する。

In [4]:
def CQGtick_csv_to_session(file_name_read):
    print(file_name_read)
    tsd=pd.read_csv(file_name_read,index_col=0,header=0,parse_dates=True)
    tsd.index.name='Date'
    indexdate=pd.unique(tsd.index.date)
    nindexdate=len(indexdate)
    tmp=pd.DataFrame([])
    da=[]
    trade=[]
    for i in range(nindexdate):
        tmp=tsd.loc[str(indexdate[i])+' 8:45:00+09:00':\
                    str(indexdate[i])+' 15:15:00+09:00']
        if not tmp.empty:
            index0=str(indexdate[i])+' 9:00'
            o=tmp.iloc[0].Price
            h=max(tmp.Price)
            l=min(tmp.Price)
            c=tmp.iloc[-1].Price
            v=sum(tmp.Volume)
            tv=sum(tmp.Volume*tmp.Price)
            da.append(index0)
            trade.append([o,h,l,c,v,tv])
            tmp=pd.DataFrame([])
        if i+1<nindexdate: 
            tmp=tsd.loc[str(indexdate[i])+' 16:30:00+09:00':\
                        str(indexdate[i+1])+' 6:00:00+09:00']
        if not tmp.empty:
            index0=str(indexdate[i])+' 16:30'
            o=tmp.iloc[0].Price
            h=max(tmp.Price)
            l=min(tmp.Price)
            c=tmp.iloc[-1].Price
            v=sum(tmp.Volume)
            tv=sum(tmp.Volume*tmp.Price)
            da.append(index0)
            trade.append([o,h,l,c,v,tv])
            tmp=pd.DataFrame([])
    return pd.DataFrame(trade,index=da,\
            columns=['Open','High','Low','Close','Volume','TradingValue'])

In [5]:
if __name__ == "__main__":
    t1=datetime.now()
    filenames2018=["n225m201812Mar.csv"
    ]
    buf_path0="C:\\users\\xxx\\documents\\Database\\n225\\tick\\"
    buf_path="C:\\users\\xxx\\documents\\Database\\n225\\session\\"
    filenames=filenames2018

    for i,filename in enumerate(filenames):
        file_name_read=buf_path0+filename
        ts=CQGtick_csv_to_session(file_name_read)
        if i==0:
            tsd=ts
        else:
            tsd=pd.concat([tsd,ts])
    file_name_save=buf_path+"nikkei225fm_2_2018.csv"
    tsd.to_csv(file_name_save)

    t2=datetime.now()
    print(t2-t1,"---------------------------------------------------------")

C:\users\hm201\documents\Database\n225\tick\n225m201812Mar.csv
0:00:26.011104 ---------------------------------------------------------
